In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify drive contents
!ls /content/drive/MyDrive/

Mounted at /content/drive
 best_resnet34_model.pth       ideas.gdoc		  resnet50_finetuned.pth
 best_resnet50_finetuned.pth  'Medicare Data Project'	  Spam
'Billing Project'	      'Portfolio Website'	 'Untitled document.gdoc'
 Budget.gsheet		      'Pre 2k25'		 'x rays project'
'Colab Notebooks'	       preprocessed_dataset.zip
 Documents		       resnet50_checkpoint.pth


In [ ]:
!unzip /content/drive/MyDrive/preprocessed_dataset.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/preprocessed_dataset/train/not fractured/4-rotated2-rotated1-rotated2 (2).jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/69-rotated2.jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/50-rotated2-rotated1-rotated1.jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/44-rotated1-rotated1-rotated1-rotated1 (1).jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/61-rotated3-rotated2-rotated3-rotated1.jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/5-rotated2-rotated1-rotated1-rotated1.jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/60-rotated3-rotated3-rotated2.jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/6-rotated3-rotated3.jpg  
  inflating: /content/preprocessed_dataset/train/not fractured/68-rotated2-rotated2-rotated2 (2).jpg  
  inflating: /content/preprocessed_dataset/train/

In [ ]:
!pip uninstall torch torchvision -y
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import torch
import torchvision
import torch.nn as nn
from torchvision import models

# ✅ Load Pretrained ResNet-50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)  # Use pretrained weights

# Modify fully connected (fc) layer: Add Dropout (prevents overfitting)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # 50% dropout
    nn.Linear(num_ftrs, 2)  # Two classes (fractured & not fractured)
)

model.to(device)
model.train()  # Set to training mode

print("✅ ResNet-50 Model Loaded!")


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 177MB/s]


✅ ResNet-50 Model Loaded!


In [ ]:
# RELOAD MODEL !
import torch
import torchvision.models as models
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Reload ResNet-50 (best pre-finetuned model)
model = models.resnet50(weights=None)  # Load architecture without pretrained weights
model.fc = nn.Linear(2048, 2)  # Ensure correct classification layer

# Load the best saved model before fine-tuning issues
checkpoint_path = "/content/drive/MyDrive/resnet50_checkpoint.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))

model.to(device)
model.eval()  # Set to evaluation mode

print("✅ Successfully reloaded the OG ResNet-50 model!")


<ipython-input-50-81ca56560fad>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=device))


✅ Successfully reloaded the OG ResNet-50 model!


In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Reload ResNet-50 (Best Model Before Fine-Tuning)
model = models.resnet50(weights=None)  # Load architecture without pretrained weights
model.fc = nn.Linear(2048, 2)  # Ensure correct classification layer

# Load the best checkpoint so far
checkpoint_path = "/content/drive/MyDrive/best_resnet50_finetuned.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))

model.to(device)
model.train()  # Set back to training mode

print("✅ Successfully reloaded the best ResNet-50 model!")


<ipython-input-36-9d321dd21925>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=device))


✅ Successfully reloaded the best ResNet-50 model!


In [ ]:
import os
print(os.listdir("/content/preprocessed_dataset/train"))
print(os.listdir("/content/preprocessed_dataset/test"))


['.DS_Store', 'not fractured', 'fractured']
['not fractured', 'fractured']


In [ ]:
import os
import cv2  # Import OpenCV (for image loading)
import torch
from torch.utils.data import Dataset, DataLoader  # Import Dataset & DataLoader
from torchvision import transforms
from PIL import Image  # ✅ Import Image for conversion


# Define transformations with augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.3),  # ✅ Low flip probability
    transforms.RandomRotation(degrees=5),  # ✅ Less aggressive rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # ✅ Small brightness changes
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])



# Modify Dataset Class to Use Transformations
class FractureDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(["not fractured", "fractured"]):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                if img_name == ".DS_Store":  # Ignore macOS system file
                    continue
                img_path = os.path.join(class_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # ✅ Convert to PIL format

        if self.transform:
            image = self.transform(image)  # ✅ Apply transform (already in PIL format)

        label = torch.tensor(label, dtype=torch.int64)
        return image, label


In [ ]:
from torch.utils.data import DataLoader

# Reload dataset
train_dataset = FractureDataset(root_dir="/content/preprocessed_dataset/train", transform=transform)
test_dataset = FractureDataset(root_dir="/content/preprocessed_dataset/test", transform=transforms.ToTensor())  # No augmentation for test set

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"✅ Loaded {len(train_dataset)} training images and {len(test_dataset)} test images.")


✅ Loaded 9243 training images and 506 test images.


In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    print(f"✅ GPU is enabled! You are using: {gpu_name}")
else:
    print("❌ GPU is NOT enabled. Go to Runtime > Change runtime type > Select GPU.")


✅ GPU is enabled! You are using: NVIDIA A100-SXM4-40GB


In [ ]:
import psutil

def get_ram_usage():
    total_memory = psutil.virtual_memory().total / 1e9  # Convert to GB
    available_memory = psutil.virtual_memory().available / 1e9  # Convert to GB
    print(f"✅ Total RAM: {total_memory:.2f} GB, Available RAM: {available_memory:.2f} GB")

get_ram_usage()


✅ Total RAM: 89.63 GB, Available RAM: 85.82 GB


In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # ✅ Ensures model weights are on the correct device

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch.optim as optim

# Define loss function (CrossEntropy for classification)
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)  # Reduce LR every 2 epochs

# Set number of epochs
num_epochs = 2 #just optimizing


In [ ]:
import torch

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"GPU Device Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU detected'}")


PyTorch Version: 2.5.1+cu124
CUDA Available: True
CUDA Version: 12.4
GPU Device Name: NVIDIA A100-SXM4-40GB


In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

239

In [ ]:
!nvidia-smi

Tue Feb 11 17:53:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             49W /  400W |   15623MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# attempted optimization
# ✅ Reduce Learning Rate for Slower Fine-Tuning
for param_group in optimizer.param_groups:
    param_group["lr"] = 8e-6  # ✅ Moderate fine-tuning learning rate

# ✅ Increase Dropout in Fully Connected Layer to Improve Generalization
model.fc = nn.Sequential(
    nn.Dropout(0.1),  # ✅ Light regularization, prevents losing useful patterns
    nn.Linear(2048, 2)
)

# ✅ Ensure the updated fc layer is moved to the correct device
model.to(device)  # ✅ Move the model & new `fc` layer to the same GPU/CPU



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import os
import torch.optim as optim
from tqdm import tqdm

# Save best checkpoint path
best_checkpoint_path = "/content/drive/MyDrive/best_resnet50_finetuned_v2.pth"

# ✅ Reset Early Stopping Counter
best_test_loss = float("inf")
patience_counter = 1

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx, (images, labels) in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

        if batch_idx % 500 == 0 and batch_idx > 0:
            torch.save(model.state_dict(), best_checkpoint_path)
            print(f"✅ Checkpoint saved at batch {batch_idx} in epoch {epoch+1}")

    scheduler.step()

    avg_train_loss = running_loss / len(train_loader)

    # 🔍 EVALUATE ON TEST DATA
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = 100 * correct / total

    print(f"✅ Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_checkpoint_path)
        print("✅ New best model saved!")
    else:
        patience_counter += 1
        print(f"⏳ No improvement. Early stopping patience left: {early_stopping_patience - patience_counter}")
        if patience_counter >= early_stopping_patience:
            print("🛑 Early stopping triggered. Training stopped.")
            break


Epoch 1/2: 100%|██████████| 289/289 [02:29<00:00,  1.93it/s, loss=0.0445]


✅ Epoch [1/2] - Train Loss: 0.2556, Test Loss: 0.4365, Accuracy: 80.83%
✅ New best model saved!


Epoch 2/2: 100%|██████████| 289/289 [02:29<00:00,  1.93it/s, loss=0.0126]


✅ Epoch [2/2] - Train Loss: 0.0289, Test Loss: 0.3699, Accuracy: 82.41%
✅ New best model saved!


In [ ]:
# training iteration 1 (resnet50_checkpoint.pth)
#🔍 Test Loss: 0.0441
#🎯 Test Accuracy: 98.42%

# training iteration 2
#🔍 Test Loss: 0.5596
#🎯 Test Accuracy: 68.77%

# reload old model iteration 3 (better generalization rather than just rememmbering) (best_resnet50_finetuned.pth)
#🔍 Test Loss: 0.3015
#🎯 Test Accuracy: 88.34%

# optimization iteration 4
#🔍 Test Loss: 0.6526
#🎯 Test Accuracy: 75.69%

# optimization interation 5
#🔍 Test Loss: 0.3699
#🎯 Test Accuracy: 82.41%


# fuck this im using my OG model.





In [ ]:
model.eval()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

avg_test_loss = test_loss / len(test_loader)
accuracy = 100 * correct / total

print(f"🔍 Test Loss: {avg_test_loss:.4f}")
print(f"🎯 Test Accuracy: {accuracy:.2f}%")


🔍 Test Loss: 0.0446
🎯 Test Accuracy: 98.42%


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import numpy as np

# ✅ Track actual labels and predictions
true_labels = []
predicted_labels = []

# ✅ No gradients needed during evaluation
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)  # Convert logits to class predictions

        true_labels.extend(labels.cpu().numpy())  # Store actual labels
        predicted_labels.extend(preds.cpu().numpy())  # Store predicted labels

# ✅ Compute Evaluation Metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average="binary")  # Binary classification
recall = recall_score(true_labels, predicted_labels, average="binary")
f1 = f1_score(true_labels, predicted_labels, average="binary")

print(f"🔍 Test Accuracy: {accuracy:.4f}")
print(f"🎯 Precision: {precision:.4f}")
print(f"📌 Recall: {recall:.4f}")
print(f"🔥 F1-Score: {f1:.4f}")


🔍 Test Accuracy: 0.9842
🎯 Precision: 1.0000
📌 Recall: 0.9664
🔥 F1-Score: 0.9829


In [ ]:
!kill -9 -1

In [ ]:
# download model
from google.colab import files

checkpoint_path = "/content/drive/MyDrive/resnet50_checkpoint.pth"
files.download(checkpoint_path)  # ✅ This will download the model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
from google.colab import files

files.download('/content/drive/MyDrive/best_resnet50_finetuned_v2.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

files.download('/content/drive/MyDrive/best_resnet50_finetuned_v2.pth')